In [1]:
import torch
import pandas as pd
from torch import nn, optim
from torch.autograd.variable import Variable
from torchvision import transforms, datasets
from data_treatment import DataSet, DataAtts
from discriminator import *
from generator import *
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import glob

import numpy as np

In [2]:
folder = widgets.Dropdown(
    options=glob.glob("models/credit_card*"),
    description='Folder:',
    # value="models/diabetes_escalonated",
    disabled=False,
)
display(folder)

Dropdown(description='Folder:', options=('models/credit_card_fraud-size0_TrainUpsampledPatient', 'models/credi…

In [3]:


folder_name = folder.value+"/generator*.pt"
model_widget = widgets.Dropdown(
    options=glob.glob(folder_name),
    description='Generator:',
    disabled=False,
)
display(model_widget)

Dropdown(description='Generator:', options=('models/credit_card_fraud-size4_TrainUpsampledPatient/generatorcre…

In [4]:
original_db_name = folder.value[7:]
# original_db_path = "original_data/" + original_db_name + ".csv"
original_db_path = "grg_credit_card/" + original_db_name + ".csv"
original_db = pd.read_csv(original_db_path)
original_db_size=original_db.shape[0]

In [5]:
try:
    checkpoint= torch.load(model_widget.value, map_location='cuda')
except:
    checkpoint= torch.load(model_widget.value, map_location='cpu')
checkpoint['model_attributes']['out_features'] = len(original_db.columns)
checkpoint['model_attributes']['hidden_layers'] = [512, 256]
generator = GeneratorNet(**checkpoint['model_attributes'])
generator.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [6]:
if not os.path.exists("fake_data/" + original_db_name ):
    os.makedirs("fake_data/" + original_db_name )

In [7]:
# size = original_db_size
# size = 200
# size = 50
num_samples = 10000
synthetic_data = generator.create_data(100*num_samples)

In [8]:

syn_labels = synthetic_data[:,-1] > 0.5
syn_classes, syn_class_count = np.unique(syn_labels, return_counts = True)
print(f"[Pre-Downsampling] syn_classes, syn_class_count = {syn_classes, syn_class_count}")

[Pre-Downsampling] syn_classes, syn_class_count = (array([False,  True]), array([504521, 495479]))


In [9]:

df = pd.DataFrame(synthetic_data, columns=original_db.columns)
#Changes the name to be easier to read
# name = model_widget.value.split("/")[-1][10:-4] + "_size-" + str(size)
name = original_db_name + "_size-" + str(num_samples)
df.to_csv( "fake_data/" + original_db_name + "/" + name + ".csv", index=False)

In [8]:
num_classes = 2

In [9]:

syn_labels = synthetic_data[:,-1] > 0.5
syn_classes, syn_class_count = np.unique(syn_labels, return_counts = True)
print(f"[Pre-Downsampling] syn_classes, syn_class_count = {syn_classes, syn_class_count}")

per_cls_sample = int(num_samples/len(syn_classes))
c_idx_list = []
for c in syn_classes:
    c_idx = np.where(syn_labels == c)[0]
    c_idx = np.random.choice(c_idx, per_cls_sample, replace = len(c_idx) < per_cls_sample)
    c_idx_list.extend(c_idx)

if len(c_idx_list) < num_samples:
    c_idx_list.extend(c_idx[:num_samples - len(c_idx_list)])

synthetic_data = synthetic_data[c_idx_list]
assert synthetic_data.shape[0] == num_samples, "Error! Selecting samples from a large batch failed."

syn_classes, syn_class_count = np.unique(synthetic_data[:, -1] > 0.5, return_counts = True)
print(f"syn_classes, syn_class_count = {syn_classes, syn_class_count}")



[Pre-Downsampling] syn_classes, syn_class_count = (array([ True]), array([1000000]))
syn_classes, syn_class_count = (array([ True]), array([10000]))


In [10]:

df = pd.DataFrame(synthetic_data, columns=original_db.columns)
#Changes the name to be easier to read
# name = model_widget.value.split("/")[-1][10:-4] + "_size-" + str(size)
name = original_db_name + "_size-" + str(num_samples)
df.to_csv( "fake_data/" + original_db_name + "/" + name + ".csv", index=False)